https://www.kdnuggets.com/2022/03/loss-functions-explainer.html

https://machinelearningmastery.com/rfe-feature-selection-in-python/

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

In [1]:
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, when

In [2]:
import numpy as np
import pandas as pd

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from IPython.display import display, HTML
import warnings

In [5]:
from sklearn.model_selection import StratifiedShuffleSplit

In [6]:
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFECV, RFE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [7]:
from sklearn.preprocessing import QuantileTransformer

In [8]:
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, d2_absolute_error_score, mean_squared_log_error
from sklearn.model_selection import ParameterGrid

In [9]:
# Linear Regression
from sklearn.linear_model import LinearRegression

# Ridge Regression
from sklearn.linear_model import Ridge

# Lasso Regression
from sklearn.linear_model import Lasso

# ElasticNet Regression
from sklearn.linear_model import ElasticNet

# Support Vector Regression (SVR)
from sklearn.svm import SVR

# Decision Tree Regression
from sklearn.tree import DecisionTreeRegressor

# Random Forest Regression
from sklearn.ensemble import RandomForestRegressor

# Gradient Boosting Regression
from sklearn.ensemble import GradientBoostingRegressor

# AdaBoost Regression
from sklearn.ensemble import AdaBoostRegressor

# XGBoost Regression
from xgboost import XGBRegressor

# LightGBM Regression
# from lightgbm import LGBMRegressor

# CatBoost Regression
# from catboost import CatBoostRegressor


In [10]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.svm import SVR, NuSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from xgboost import XGBRegressor

In [11]:
from explainerdashboard import RegressionExplainer, ExplainerDashboard

/home/konradballegro/.local/share/virtualenvs/konradballegro-pfUEMlPh/lib/python3.8/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/konradballegro/.local/share/virtualenvs/konradballegro-pfUEMlPh/lib/python3.8/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mo

In [12]:
display(HTML("<style>.container { width:100% !important; }</style>"))
warnings.filterwarnings('ignore')
# pd.options.display.float_format = "{:.2f}".format

In [13]:
pyspark.__version__

'3.3.2'

In [14]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('regression') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/24 16:24:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "True") \
    .csv("/home/konradballegro/orchestration/data/inputs/offers.csv")

In [17]:
# Count the number of rows in the DataFrame
num_rows = df.count()

# Count the number of columns in the DataFrame
num_cols = len(df.columns)

# Print the shape of the DataFrame
print("Number of rows: ", num_rows)
print("Number of columns: ", num_cols)

Number of rows:  129589
Number of columns:  230


In [18]:
headers = df.columns
for h, head in enumerate(headers):
    print(f"{h}: {head}")

0: Offer from
1: Category
2: Show offers with VIN number
3: Has registration number
4: Vehicle brand
5: Vehicle model
6: Version
7: Generation
8: Year of production
9: Mileage
10: Engine capacity
11: Fuel type
12: Power
13: Gearbox
14: Range
15: Drive
16: Battery capacity
17: Battery ownership type
18: CO2 emissions
19: Particulate filter
20: City fuel consumption
21: Body type
22: Number of doors
23: Number of seats
24: Color
25: Metallic
26: Color type
27: Right-hand drive (Anglik)
28: Country of origin
29: Leasing
30: VAT margin
31: VAT invoice
32: Manufacturer warranty period
33: Financing possibility
34: First registration
35: Registered in Poland
36: First owner
37: Accident-free
38: Serviced at authorized service center
39: Condition
40: ABS
41: Apple CarPlay
42: Android Auto
43: Rear side airbags43
44: Driver side airbag
45: CD
46: Central locking
47: Electric front windows
48: Electrically adjustable passenger seat
49: Electrically adjustable mirrors
50: Immobilizer
51: Driver

In [19]:
# Sample 20% of the rows without replacement
sampled_df = df.sample(withReplacement=False, fraction=0.2)

In [20]:
# Count the number of rows in the DataFrame
num_rows = sampled_df.count()

# Count the number of columns in the DataFrame
num_cols = len(sampled_df.columns)

# Print the shape of the DataFrame
print("Number of rows: ", num_rows)
print("Number of columns: ", num_cols)

Number of rows:  25925
Number of columns:  230


In [21]:
price = sampled_df.select("Price").rdd.flatMap(lambda x: x).collect()

In [22]:
# # Assuming you have a PySpark DataFrame called 'sampled_df' with a column 'Price'
# # Extract the 'Price' column as a list
# price_data = sampled_df.select("Price").rdd.flatMap(lambda x: x).collect()

# # Create a histogram using Seaborn
# sns.histplot(data=price_data, bins=10, kde=True)

# # Customize the plot
# plt.xlabel("Price")
# plt.ylabel("Frequency")
# plt.title("Histogram of Prices")

# # Show the plot
# plt.show()


In [23]:
df = df.filter((df["Currency"] == "PLN") &
                        (df["Country of origin"] == "Polska") &
                        (df["Accident-free"].isNotNull()) &
                        (df["Price"].isNotNull()) &
                        (df["Offer from"].isNotNull()) &
                        (df["Condition"].isNotNull()) &
                        (df["Condition"] == 'Używane') &
                        (df["Vehicle brand"].isNotNull()) &
#                         (df["Vehicle model"].isNotNull()) &
                        (df["Year of production"].isNotNull()) &
                        (df["Mileage"].isNotNull()) &
                        (df["Fuel type"].isNotNull()) &
                        (df["Power"].isNotNull()) &
                        (df["Gearbox"].isNotNull()) &
                        (df["Body type"].isNotNull()) &
                        (df["Number of doors"].isNotNull())
                       ).select(col("Price").cast("float").alias("Price"),
                                "Offer from",
                                "Condition",
                                "Vehicle brand",
                                "Vehicle model",
                                col("Year of production").cast("string").alias("Year of production"),
                                regexp_replace(regexp_replace(col("Mileage"), " ", ""), "km", "").cast("float").alias("Mileage"),
                                "Fuel type",
                                regexp_replace(regexp_replace(col("Power"), " ", ""), "KM", "").cast("integer").alias("Power"),
                                "Gearbox",
                                "Body type",
                                regexp_replace(col("Number of doors"), "\\.0$", "").alias("Number of doors"),
                                "URL path",
                                "ID",
                                "Epoch"
                               )

In [24]:
# Count the number of rows in the DataFrame
num_rows = df.count()

# Count the number of columns in the DataFrame
num_cols = len(df.columns)

# Print the shape of the DataFrame
print("Number of rows: ", num_rows)
print("Number of columns: ", num_cols)

Number of rows:  26048
Number of columns:  15


In [25]:
df.show()

+--------+---------------+---------+-------------+-------------+------------------+--------+-----------+-----+------------+-------------+---------------+--------------------+--------------------+----------+
|   Price|     Offer from|Condition|Vehicle brand|Vehicle model|Year of production| Mileage|  Fuel type|Power|     Gearbox|    Body type|Number of doors|            URL path|                  ID|     Epoch|
+--------+---------------+---------+-------------+-------------+------------------+--------+-----------+-----+------------+-------------+---------------+--------------------+--------------------+----------+
| 28000.0|Osoby prywatnej|  Używane|        Honda|        Civic|              2010|189347.0|    Benzyna|  140|    Manualna|        Sedan|              4|https://www.otomo...|000d1349f23a1d685...|1687241613|
| 92900.0|Osoby prywatnej|  Używane|        Honda|        Civic|              2018| 81240.0|    Benzyna|  182|    Manualna|      Kompakt|              5|https://www.otomo..

In [26]:
distinct_offers = df.select("Offer from").distinct().rdd.flatMap(lambda x: x).collect()
distinct_conditions = df.select("Condition").distinct().rdd.flatMap(lambda x: x).collect()
distinct_brands = df.select("Vehicle brand").distinct().rdd.flatMap(lambda x: x).collect()
# distinct_models = df.select("Vehicle model").distinct().rdd.flatMap(lambda x: x).collect()
distinct_years = df.select("Year of production").distinct().rdd.flatMap(lambda x: x).collect()
distinct_fuel = df.select("Fuel type").distinct().rdd.flatMap(lambda x: x).collect()
distinct_gearbox = df.select("Gearbox").distinct().rdd.flatMap(lambda x: x).collect()
distinct_body = df.select("Body type").distinct().rdd.flatMap(lambda x: x).collect()
distinct_doors = df.select("Number of doors").distinct().rdd.flatMap(lambda x: x).collect()

In [27]:
for offer in distinct_offers:
    column_name = "Offer_type_" + offer.replace(" ", "_")
    df = df.withColumn(column_name, when(df["Offer from"] == offer, 1).otherwise(0))

In [28]:
for condition in distinct_conditions:
    column_name = "Condition_" + condition.replace(" ", "_")
    df = df.withColumn(column_name, when(df["Condition"] == condition, 1).otherwise(0))

In [29]:
for brand in distinct_brands:
    column_name = "Vehicle_brand_" + brand.replace(" ", "_")
    df = df.withColumn(column_name, when(df["Vehicle brand"] == brand, 1).otherwise(0))

In [30]:
# for model in distinct_models:
#     column_name = "Vehicle_model_" + model.replace(" ", "_")
#     df = df.withColumn(column_name, when(df["Vehicle model"] == model, 1).otherwise(0))

In [31]:
for year in distinct_years:
    column_name = "Year_of_production_" + str(year)
    df = df.withColumn(column_name, when(df["Year of production"] == year, 1).otherwise(0))

In [32]:
for fuel in distinct_fuel:
    column_name = "Fuel_type_" + fuel.replace(" ", "_")
    df = df.withColumn(column_name, when(df["Fuel type"] == fuel, 1).otherwise(0))

In [33]:
for gearbox in distinct_gearbox:
    column_name = "Gearbox_" + gearbox.replace(" ", "_")
    df = df.withColumn(column_name, when(df["Gearbox"] == gearbox, 1).otherwise(0))

In [34]:
for body in distinct_body:
    column_name = "Body_type_" + body.replace(" ", "_")
    df = df.withColumn(column_name, when(df["Body type"] == body, 1).otherwise(0))

In [35]:
for doors in distinct_doors:
    column_name = "Number_of_doors_" + str(doors)
    df = df.withColumn(column_name, when(df["Number of doors"] == doors, 1).otherwise(0))

In [36]:
# Assuming your DataFrame is called df
columns_to_drop = ["Offer from", "Condition", "Vehicle brand", "Vehicle model", "Year of production", "Fuel type", "Gearbox", "Body type", "Number of doors", "URL path", "ID", "Epoch"]

# Drop the specified columns
df = df.drop(*columns_to_drop)

In [37]:
df = df.filter(df["Price"].isNotNull())

In [38]:
df_pandas = df.toPandas()

23/06/24 16:25:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [39]:
df_pandas

,Price,Mileage,Power,Offer_type_Osoby_prywatnej,Offer_type_Firmy,Condition_Używane,Vehicle_brand_Infiniti,Vehicle_brand_Lexus,Vehicle_brand_Jaguar,Vehicle_brand_Maserati,...,Body_type_Coupe,Body_type_Minivan,Body_type_Kompakt,Body_type_Auta_miejskie,Body_type_Kombi,Number_of_doors_3,Number_of_doors_5,Number_of_doors_4,Number_of_doors_2,Number_of_doors_6
0,28000.0,189347.0,140,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,92900.0,81240.0,182,1,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,89000.0,86500.0,155,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,48900.0,218350.0,150,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,7500.0,245000.0,90,1,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26028,129900.0,17929.0,150,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
26029,185900.0,41800.0,310,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
26030,259000.0,3000.0,252,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
26031,359283.0,6200.0,292,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [40]:
df_pandas.isnull().sum()

Price                         0
Mileage                       0
Power                         0
Offer_type_Osoby_prywatnej    0
Offer_type_Firmy              0
                             ..
Number_of_doors_3             0
Number_of_doors_5             0
Number_of_doors_4             0
Number_of_doors_2             0
Number_of_doors_6             0
Length: 126, dtype: int64

In [41]:
df_pandas.to_csv('/home/konradballegro/notebooks/outputs/data/offers.csv')

In [42]:
y = df_pandas["Price"]
y

0         28000.0
1         92900.0
2         89000.0
3         48900.0
4          7500.0
           ...   
26028    129900.0
26029    185900.0
26030    259000.0
26031    359283.0
26032    268000.0
Name: Price, Length: 26033, dtype: float32

In [43]:
# Convert the pandas Series to a NumPy array and reshape it
y_array = y.to_numpy().reshape(-1, 1)

# Quantile transformation
transformer = QuantileTransformer(output_distribution='normal')
y_transformed = transformer.fit_transform(y_array)

In [44]:
y_transformed

array([[-1.2982711 ],
       [ 0.15619943],
       [ 0.08289633],
       ...,
       [ 1.4827601 ],
       [ 1.7900579 ],
       [ 1.5189546 ]], dtype=float32)

In [45]:
y = y_transformed

In [46]:
y.max()

5.1993375

In [47]:
y.min()

-5.1993375

In [48]:
X = df_pandas.loc[:, ~df_pandas.columns.isin(["Price"])]
X.head()

,Mileage,Power,Offer_type_Osoby_prywatnej,Offer_type_Firmy,Condition_Używane,Vehicle_brand_Infiniti,Vehicle_brand_Lexus,Vehicle_brand_Jaguar,Vehicle_brand_Maserati,Vehicle_brand_Jeep,...,Body_type_Coupe,Body_type_Minivan,Body_type_Kompakt,Body_type_Auta_miejskie,Body_type_Kombi,Number_of_doors_3,Number_of_doors_5,Number_of_doors_4,Number_of_doors_2,Number_of_doors_6
0,189347.0,140,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,81240.0,182,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,86500.0,155,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,218350.0,150,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,245000.0,90,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


In [49]:
train_size = 0.8

In [50]:
from sklearn.model_selection import train_test_split

# Split data into train and remaining data
X_train, X_remain, y_train, y_remain = train_test_split(X, y, test_size=0.3, random_state=42)

# Split remaining data into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_remain, y_remain, test_size=0.5, random_state=42)


In [51]:
print("Shape of X_train:", X_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (18223, 125)
Shape of X_val: (3905, 125)
Shape of X_test: (3905, 125)
Shape of y_train: (18223, 1)
Shape of y_val: (3905, 1)
Shape of y_test: (3905, 1)


In [52]:
corr_matrix = X_train.corr(method = "spearman").abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
X_train = X_train.drop(to_drop, axis = 1)
X_val = X_val.drop(to_drop, axis = 1)
X_test = X_test.drop(to_drop, axis = 1)

In [53]:
X_train

,Mileage,Power,Offer_type_Osoby_prywatnej,Condition_Używane,Vehicle_brand_Infiniti,Vehicle_brand_Lexus,Vehicle_brand_Jaguar,Vehicle_brand_Maserati,Vehicle_brand_Jeep,Vehicle_brand_Lancia,...,Body_type_Coupe,Body_type_Minivan,Body_type_Kompakt,Body_type_Auta_miejskie,Body_type_Kombi,Number_of_doors_3,Number_of_doors_5,Number_of_doors_4,Number_of_doors_2,Number_of_doors_6
22988,327550.0,160,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
22826,74100.0,150,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
18917,48000.0,111,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
14723,172000.0,86,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
6775,11302.0,330,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,217000.0,111,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
5390,4600.0,254,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
860,64274.0,84,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
15795,300000.0,140,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [54]:
y_train

array([[-1.0487477 ],
       [ 0.1931432 ],
       [-0.15365991],
       ...,
       [-0.52641684],
       [-0.9917463 ],
       [-0.76470965]], dtype=float32)

In [53]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming df is your DataFrame containing the data
subset = X_train[["Mileage", "Power"]]

scaler = StandardScaler()
scaler.fit(subset)
subset_scaled = scaler.transform(subset)

X_train["Mileage"] = subset_scaled[:, 0]
X_train["Power"] = subset_scaled[:, 1]

In [54]:
X_train

,Mileage,Power,Offer_type_Osoby_prywatnej,Condition_Używane,Vehicle_brand_Infiniti,Vehicle_brand_Lexus,Vehicle_brand_Jaguar,Vehicle_brand_Maserati,Vehicle_brand_Jeep,Vehicle_brand_Lancia,...,Body_type_Coupe,Body_type_Minivan,Body_type_Kompakt,Body_type_Auta_miejskie,Body_type_Kombi,Number_of_doors_3,Number_of_doors_5,Number_of_doors_4,Number_of_doors_2,Number_of_doors_6
22988,2.673892,-0.055748,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
22826,-0.362372,-0.170994,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
18917,-0.675043,-0.620456,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
14723,0.810444,-0.908572,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
6775,-1.114675,1.903444,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,1.349532,-0.620456,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
5390,-1.194963,1.027570,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
860,-0.480085,-0.931622,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
15795,2.343850,-0.286241,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


neg_mean_absolute_error

In [55]:
selected_features = [
    'Mileage',
    'Power',
    'Offer_type_Osoby_prywatnej',
#     'Vehicle_brand_Infiniti',
#     'Vehicle_brand_Lexus',
#     'Vehicle_brand_Jaguar',
#     'Vehicle_brand_Maserati',
#     'Vehicle_brand_Jeep',
#     'Vehicle_brand_Lancia',
#     'Vehicle_brand_Kia',
#     'Vehicle_brand_Hyundai',
#     'Vehicle_brand_Honda',
#     'Vehicle_brand_Lamborghini',
#     'Vehicle_brand_Ligier',
#     'Vehicle_brand_Isuzu',
#     'Vehicle_brand_Land_Rover',
    'Vehicle_brand_Mercedes-Benz',
#     'Vehicle_brand_McLaren',
    'Gearbox_Manualna',
    'Number_of_doors_2',
    'Number_of_doors_5',
#     'Number_of_doors_6',
    'Fuel_type_Benzyna',
    'Fuel_type_Benzyna+LPG',
    'Fuel_type_Hybryda',
#     'Fuel_type_Elektryczny',
    'Body_type_SUV',
    'Body_type_Minivan',
    'Body_type_Coupe'
]

In [56]:
selected_features

['Mileage',
 'Power',
 'Offer_type_Osoby_prywatnej',
 'Vehicle_brand_Mercedes-Benz',
 'Gearbox_Manualna',
 'Number_of_doors_2',
 'Number_of_doors_5',
 'Fuel_type_Benzyna',
 'Fuel_type_Benzyna+LPG',
 'Fuel_type_Hybryda',
 'Body_type_SUV',
 'Body_type_Minivan',
 'Body_type_Coupe']

In [57]:
X_train_copy = X_train.copy()

In [58]:
X_train = X_train[selected_features]
X_val = X_val[selected_features]
X_test = X_test[selected_features]

In [59]:
X_train

,Mileage,Power,Offer_type_Osoby_prywatnej,Vehicle_brand_Mercedes-Benz,Gearbox_Manualna,Number_of_doors_2,Number_of_doors_5,Fuel_type_Benzyna,Fuel_type_Benzyna+LPG,Fuel_type_Hybryda,Body_type_SUV,Body_type_Minivan,Body_type_Coupe
22988,2.673892,-0.055748,1,0,0,0,0,1,0,0,0,0,0
22826,-0.362372,-0.170994,0,0,0,0,0,0,0,0,0,0,0
18917,-0.675043,-0.620456,0,0,1,0,1,1,0,0,1,0,0
14723,0.810444,-0.908572,1,0,1,0,1,1,0,0,0,0,0
6775,-1.114675,1.903444,0,1,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,1.349532,-0.620456,0,0,1,0,1,0,1,0,0,0,0
5390,-1.194963,1.027570,1,0,0,0,1,0,0,0,1,0,0
860,-0.480085,-0.931622,0,0,1,0,1,1,0,0,0,0,0
15795,2.343850,-0.286241,1,0,1,0,1,1,0,0,0,0,0


### XGB

In [60]:
model = XGBRegressor()
model.load_model('/home/konradballegro/notebooks/outputs/models/xgb_model.model')

In [61]:
# Use the loaded model for predictions
y_pred = model.predict(X_test)

In [62]:
# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.6788684


In [63]:
# Evaluate the model using mean squared error
r2 = r2_score(y_test, y_pred)
print("R2:", r2)

R2: 0.32754705189235556


In [64]:
explainer = RegressionExplainer(model, X_test, y_test, n_jobs=-1)

Changing class type to XGBRegressionExplainer...
Generating self.shap_explainer = shap.TreeExplainer(model)


In [65]:
db = ExplainerDashboard(explainer, 
                        title="Otomoto", # defaults to "Model Explainer"
                        shap_interaction=False, # you can switch off tabs with bools
                        )

Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating shap values...
Calculating predictions...
Calculating residuals...
Calculating absolute residuals...
Generating xgboost model dump...
Calculating dependencies...
Calculating importances...
Calculating ShadowDecTree for each individual decision tree...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...


In [66]:
db.save_html('/home/konradballegro/notebooks/outputs/reports/xgb_explainer.html')